# **Implementation of the Genetic Algorithm**

El siguiente algoritmo genético nos dará las dimensiones de una piscina rectangular, cuyo volumen es 2500 m3, para que sea recubierta por la menor cantidad de azulejos posibles. 
La superficie de cada azulejo es 0.5 m2 y el volumen de la piscina puede variar un rango de 10%. 

## Libraries

In [429]:
import random
import numpy

## GA process

In [430]:
def genetic_algorithm(optimization, max_size, ind_size, k_tournament, prob_crossover, prob_mutation, limit_gen, limit_value=None):
    generation = 0
    population = initial(max_size, ind_size)
    fit_pops = evaluate(population)
    while not check_stop(fit_pops, generation, optimization, limit_gen, limit_value):
        new_fit_pops = execute_generation(fit_pops, max_size, k_tournament, prob_crossover, prob_mutation, optimization)
        fit_pops = update(fit_pops, new_fit_pops, optimization)
        generation += 1
    if optimization == 'MIN':
        best = list(sorted(fit_pops, reverse=False))[0]
    else:
        best = list(sorted(fit_pops, reverse=True))[0]
    return best

## Termination function

In [431]:
def check_stop(fits_populations, generation, optimization, limit_gen, limit_value):
    fits = [f for f, ch in fits_populations]
    if optimization == 'MIN':
        best = min(fits)
        worst = max(fits)
        reverse = False
    else:
        best = max(fits)
        worst = min(fits)
        reverse = True
    if generation % 10 == 0 or best==0:
        best_match = list(sorted(fits_populations, reverse=reverse))[0][1]
        ave = sum(fits) / len(fits)
        print(
            "[G %3d] score=(%4d, %4d, %4d): %r" %
            (generation, best, ave, worst, best_match))
        pass
    return generation >= limit_gen or (limit_value is not None and best == limit_value)


## Process of a generation

In [432]:
def execute_generation(pop, max_size, k_tournament, prob_crossover, prob_mutation, optimization):
    parents_selected = []
    if max_size > len(pop):
        max_size = len(pop)
    parents_selected = tournament(pop, max_size, k_tournament, optimization)
    i = 0
    new_pop = []
    while i < len(parents_selected):
        if random.random() < prob_crossover:
            children = crossover([parents_selected[i], parents_selected[i+1]])
        else:
            children = [parents_selected[i], parents_selected[i+1]]
        for ch in children:
            new_child = mutation(ch, prob_mutation)
            new_pop.append(new_child)
        i = i + 2
    fits_pop = evaluate(new_pop) 
    return fits_pop

## Individual initialization

In [433]:
def initial(max_size, ind_size):
    return [random_chromo(ind_size) for j in range(max_size)]

## Individual Evaluation

In [434]:
def evaluate(pop):
    #The individual is converted to a tuple where 
    #the first element is its fitness and 
    #the second one its chromosome 
    fit_pop = [(fitness(ch),  ch) for ch in pop]
    return fit_pop

## Tournament selection

In [435]:
def tournament(fits_populations, max_size, k_tournament, optimization):
    parents_selected = []
    for i in range(0, max_size):
        parents_selected.append(select_random(fits_populations, k_tournament, optimization)[1])
    return parents_selected

## Select best from random set

In [436]:
def select_random(fits_population_par, k_tournament, optimization):
    fits_population = fits_population_par.copy()
    best = None
    size = int(len(fits_population) * k_tournament)
    selected = []
    for e in range(0, size):
        i = random.randint(0, len(fits_population) - 1)
        selected.append(fits_population[i])
    
    if optimization == 'MIN':
        best = list(sorted(selected, reverse=False))[0]
    else:
        best = list(sorted(selected, reverse=True))[0]
    return best

## Crossover

### One-point crossover

In [437]:
def crossover(parents):
    father, mother = parents
    index1 = random.randint(1, len(father) - 1)
    child1 = father[:index1] + mother[index1:]
    child2 = mother[:index1] + father[index1:]
    return (child1, child2)

## Replacement of the population

### Elitism of the best solution

In [438]:
def update(population, new_population, optimization):
    if optimization == 'MIN':
        new_population = sorted(new_population, reverse=False)
        population = sorted(population, reverse=True)
    else:
        new_population = sorted(new_population, reverse=True)
        population = sorted(population, reverse=False)
    new_population[-1] = population[0]

    return new_population

# **Customization for Exercise 5**

In [439]:
pool_volume = 2500
piece_area = 0.5
error_range = 0.1

## Condition function

In [456]:
def constraint(chromo):
    for i in chromo:
        if (i <= 1): return False # Considero que una magnitud sea de al menos 1 metro.
        if (i%0.5 != 0): return False # Uso de azulejos enteros
    if (numpy.prod(chromo) < pool_volume*(1 - error_range) or numpy.prod(chromo) > pool_volume*(1 + error_range)): return False
    else: return True


## Fitness function

In [457]:
def fitness(chromo): 
    if constraint(chromo) is True:
        a = chromo[0]
        b = chromo[1]
        c = chromo[2]
        return round((2*a*(b+c) + b*c)/piece_area, 0)
    else:
        return 9999

## Random initialization of an individual

In [462]:
def random_chromo(size):
    return [random.randint(0, 100) for i in range(0, size)] # 100 metros máximo por lado

## Mutation

In [466]:
def mutation(chromosome, prob_mutation):
    d = random.randint(-1,1)
    for i in range(len(chromosome)):
        if (random.random() < prob_mutation): chromosome[i] = chromosome[i] + d*piece_area
    return chromosome

## Execution of the genetic algorithm

In [475]:
best = genetic_algorithm(optimization="MIN", max_size=400, ind_size=3, limit_gen=100, 
                  k_tournament=1, prob_crossover=0.9, prob_mutation=0.3, limit_value=1000)
print("Best Individual:", best[1])
print("Fitness:", best[0])

[G   0] score=(4032, 9973, 9999): [36, 24, 3]
[G  10] score=(2494, 6004, 9999): [24.0, 19.0, 5.0]
[G  20] score=(1870, 5661, 9999): [18.0, 12.5, 10.0]
[G  30] score=(1650, 3690, 9999): [10.0, 15.0, 15.0]
[G  40] score=(1650, 3688, 9999): [10.0, 15.0, 15.0]
[G  50] score=(1650, 3581, 9999): [10.0, 15.0, 15.0]
[G  60] score=(1650, 3500, 9999): [10.0, 15.0, 15.0]
[G  70] score=(1650, 3812, 9999): [10.0, 15.0, 15.0]
[G  80] score=(1650, 3481, 9999): [10.0, 15.0, 15.0]
[G  90] score=(1650, 3604, 9999): [10.0, 15.0, 15.0]
[G 100] score=(1650, 3584, 9999): [10.0, 15.0, 15.0]
Best Individual: [10.0, 15.0, 15.0]
Fitness: 1650.0
